## Sales Prediction using a Machine Learning Algorithm(s)

In [49]:
import pickle
import mlflow
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error
from time import gmtime, strftime
import matplotlib.pyplot as plt
import datetime

In [50]:
import os, sys
import warnings
sys.path.append(os.path.abspath(os.path.join('../scripts')))
warnings.filterwarnings('ignore')

from preprocessing import Preprocessing
from plot import Plot
import dvc.api

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle

In [51]:
# instantiate Preprocessing object
preprocess = Preprocessing()

# instantiate Plot object
plot = Plot()

In [52]:
# Get url from dvc
path = 'data/cleaned_train.csv'
repo = 'https://github.com/YohansSamuel/pharmaceutical_sales_prediction'
version = 'cleaned-v'

# Load data from dvc 
# data_url = dvc.api.get_url(path=path,repo=repo,rev=version)

In [53]:
# load the cleaned dataset
df_train = preprocess.read_csv('../data/cleaned_train.csv')
#df_train.set_index('Date', inplace=True)

file read as csv


In [54]:
df_train.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,...,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType,Decile_rank
0,1,1,0.0,0.0,0,0,a,1,1270.0,9.0,...,2013,1,1,1,1,0,1,0,2,1
1,1,2,5530.0,668.0,1,0,0,1,1270.0,9.0,...,2013,1,2,2,1,0,1,0,2,1
2,1,3,4327.0,578.0,1,0,0,1,1270.0,9.0,...,2013,1,3,3,1,0,1,0,2,1
3,1,4,4486.0,619.0,1,0,0,1,1270.0,9.0,...,2013,1,4,4,1,0,1,0,2,1
4,1,5,4997.0,635.0,1,0,0,1,1270.0,9.0,...,2013,1,5,5,1,1,1,0,2,1


In [55]:
df_train.drop(['StateHoliday'], axis=1, inplace=True)

df_train.dtypes

Store                          int64
DayOfWeek                      int64
Sales                        float64
Customers                    float64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
Year                           int64
Month                          int64
Day                            int64
weekday                        int64
weekofyear                     int64
weekend                        int64
PromoInterval                  int64
Assortment                     int64
StoreType                      int64
Decile_rank                    int64
dtype: object

In [56]:
# remove rows that have closed stores and sales = 0
df_train = df_train[df_train['Open'] == 1]
df_train = df_train[df_train['Sales'] > 0.0]

In [57]:
# data split
X = df_train.drop('Sales',axis=1)
y = df_train['Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [58]:
data_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])

pipeline = Pipeline(steps = [
               ('preprocessor', data_transformer)
              ,('regressor',RandomForestRegressor(max_depth=64, random_state=0))
           ])

In [59]:
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'a'

In [ ]:
rf_model.score(X_test, y_test)

In [ ]:
# Save model based on time st
rf_model_path = '../models/' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl'
pickle.dump(rf_model, open(rf_model_path, 'wb'))